In [6]:
#!pip install protobuf


In [7]:
#!pip install gtfs-realtime-python
#!pip install requests



## Ce script récupère en temps réel les TripUpdate

| Champ                           | Supporté | Commentaires                                      |
|----------------------------------|----------|---------------------------------------------------|
| trip (TripDescriptor)           |          |                                                   |
| trip_id                         | Oui      |                                                   |
| route_id                        | Oui      |                                                   |
| direction_id                    | Oui      | S’il est disponible dans le flux statique.        |
| start_time                      | Oui      |                                                   |
| start_date                      | Oui      |                                                   |
| schedule_relationship (ScheduleRelationship) | Oui  | Les valeurs prises en charge sont les suivantes : PRÉVUE, ANNULÉ |
| véhicule (Descripteur de véhicule) | Oui    |                                                   |
| id                               | Oui      |                                                   |
| étiquette                        | Oui      |                                                   |
| trip_headsign si disponible dans le flux statique. | Oui |                                                   |
| license_plate                   | Oui      | Disponibilité non garantie.                       |
| stop_time_update (StopTimeUpdate) | Oui     |                                                   |
| stop_sequence                   | Non      |                                                   |
| stop_id                         | Oui      |                                                   |
| arrivée (StopTimeEvent)         | Oui      |                                                   |
| retarder                        | Non      |                                                   |
| Heure                           | Oui      |                                                   |
| incertitude                     | Non      |                                                   |
| départ (StopTimeEvent)          | Oui      |                                                   |
| retarder                        | Non      |                                                   |
| Heure                           | Oui      |                                                   |
| incertitude                     | Non      |                                                   |
| schedule_relationship (ScheduleRelationship) | Non  | Les valeurs prises en charge sont les suivantes : PRÉVUE, NO_DATA si l’arrêt est fermé, Écrémé en cas d’annulation partielle du voyage, NO_DATA et les arrêts SKIPPED ne peuvent apparaître que dans les trajets en cours |
| horodatage                      | Oui      |                                                   |
| retarder                        | Non      |                                                   |


In [10]:
import os
import base64
import requests
import json
from google.transit import gtfs_realtime_pb2  
from google.protobuf.json_format import MessageToJson
from kafka import KafkaProducer

os.environ["CLIENT_ID"] = "6722072041100899" 
os.environ["CLIENT_SECRET"] = "ISo0oqPMkfCB8wvaPSxrfj20EmnuvKwE" 

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

if not client_id or not client_secret:
    print("Error: CLIENT_ID and CLIENT_SECRET must be set as environment variables.")
    exit()

auth_string = f"{client_id}:{client_secret}"
encoded_auth = base64.b64encode(auth_string.encode()).decode()

city = "tampere"  
endpoint = "tripupdate"  
url = f"https://data.waltti.fi/{city}/api/gtfsrealtime/v1.0/feed/{endpoint}"

headers = {
    "Authorization": f"Basic {encoded_auth}",
}

kafka_config = {
    'bootstrap_servers': 'localhost:9092',
}
producer = KafkaProducer(
    **kafka_config,
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

kafka_topic = "tripupdate"  

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(response.content)
    feed_json = json.loads(MessageToJson(feed))

    if feed.entity:
        first_entity = feed.entity[0]
        vehicle_json = json.loads(MessageToJson(first_entity))
        print(json.dumps(vehicle_json, indent=4))
    else:
        print("Aucune entité trouvée.")

    for entity in feed.entity:
        vehicle_json = json.loads(MessageToJson(entity))
        producer.send(kafka_topic, vehicle_json)

    producer.flush()

except requests.exceptions.RequestException as e:
    print(json.dumps({"error": f"Error connecting to the API: {e}"}, indent=4))
except Exception as e:
    print(json.dumps({"error": f"Unexpected error: {e}"}, indent=4))
finally:
    producer.close()

{
    "id": "76_14835_9017305_2025-01-10_6_1_24:05:00",
    "tripUpdate": {
        "trip": {
            "tripId": "76_14835_9017305",
            "startTime": "24:05:00",
            "startDate": "20250110",
            "scheduleRelationship": "SCHEDULED",
            "routeId": "610299",
            "directionId": 1
        },
        "stopTimeUpdate": [
            {
                "stopSequence": 17,
                "arrival": {
                    "time": "1736547885"
                },
                "departure": {
                    "time": "1736547889"
                },
                "stopId": "5132",
                "scheduleRelationship": "SCHEDULED"
            },
            {
                "stopSequence": 18,
                "arrival": {
                    "time": "1736547947"
                },
                "departure": {
                    "time": "1736547972"
                },
                "stopId": "4578",
                "scheduleRelationship": "SCHE

### Ce script récupère en temps réel les positions des véhicules de transport public à Tampere via l'endpoint vehicleposition de l'API GTFS Realtime de Waltti. 

| Champ                          | Supporté | Commentaires                                                                 |
|---------------------------------|----------|-----------------------------------------------------------------------------|
| trip (TripDescriptor)          |          |                                                                             |
| trip_id                         | Oui      |                                                                             |
| route_id                        | Oui      |                                                                             |
| direction_id                    | Oui      | S’il est disponible dans le flux statique.                                  |
| start_time                      | Oui      |                                                                             |
| start_date                      | Oui      |                                                                             |
| schedule_relationship (ScheduleRelationship) | Oui | Les valeurs prises en charge sont les suivantes : PRÉVUE                     |
| véhicule (Descripteur de véhicule) | Oui    |                                                                             |
| id                               | Oui      |                                                                             |
| étiquette                        | Oui      |                                                                             |
| trip_headsign si disponible dans le flux statique. | Oui |                                                                             |
| license_plate                   | Oui      | Disponibilité non garantie.                                                 |
| position (Position)            | Oui      |                                                                             |
| latitude                        | Oui      |                                                                             |
| longitude                       | Oui      |                                                                             |
| Attitude                        | Oui      |                                                                             |
| odomètre                        | Non      |                                                                             |
| vitesse                         | Oui      |                                                                             |
| current_stop_sequence           | Oui      |                                                                             |
| stop_id                         | Oui      |                                                                             |
| current_status                  | Oui      |                                                                             |
| horodatage                      | Oui      |                                                                             |
| congestion_level (CongestionLevel) | Oui    | La valeur prise en charge est la suivante : UNKNOWN_CONGESTION_LEVEL        |
| occupancy_status (OccupancyStatus) | Non    |                                                                             |


In [9]:
import os
import base64
import requests
import json
from google.transit import gtfs_realtime_pb2  
from google.protobuf.json_format import MessageToJson
from kafka import KafkaProducer

os.environ["CLIENT_ID"] = "6722072041100899" 
os.environ["CLIENT_SECRET"] = "ISo0oqPMkfCB8wvaPSxrfj20EmnuvKwE" 

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

if not client_id or not client_secret:
    print("Error: CLIENT_ID and CLIENT_SECRET must be set as environment variables.")
    exit()

auth_string = f"{client_id}:{client_secret}"
encoded_auth = base64.b64encode(auth_string.encode()).decode()

city = "tampere"  
endpoint = "vehicleposition"  
url = f"https://data.waltti.fi/{city}/api/gtfsrealtime/v1.0/feed/{endpoint}"

headers = {
    "Authorization": f"Basic {encoded_auth}",
}

kafka_config = {
    'bootstrap_servers': 'localhost:9092',
}
producer = KafkaProducer(
    **kafka_config,
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

kafka_topic = "vehicleposition"  

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(response.content)
    feed_json = json.loads(MessageToJson(feed))

    if feed.entity:
        first_entity = feed.entity[0]
        vehicle_json = json.loads(MessageToJson(first_entity))
        print(json.dumps(vehicle_json, indent=4))
    else:
        print("Aucune entité trouvée.")

    for entity in feed.entity:
        vehicle_json = json.loads(MessageToJson(entity))
        producer.send(kafka_topic, vehicle_json)

    producer.flush()

except requests.exceptions.RequestException as e:
    print(json.dumps({"error": f"Error connecting to the API: {e}"}, indent=4))
except Exception as e:
    print(json.dumps({"error": f"Unexpected error: {e}"}, indent=4))
finally:
    producer.close()

{
    "id": "6990_31",
    "vehicle": {
        "trip": {
            "tripId": "76_14851_9182341",
            "startTime": "21:40:00",
            "startDate": "20250110",
            "scheduleRelationship": "SCHEDULED",
            "routeId": "136990",
            "directionId": 1
        },
        "position": {
            "latitude": 61.46433,
            "longitude": 23.82826,
            "bearing": 139.0,
            "speed": 4.2222223
        },
        "currentStopSequence": 34,
        "currentStatus": "IN_TRANSIT_TO",
        "timestamp": "1736540480",
        "congestionLevel": "UNKNOWN_CONGESTION_LEVEL",
        "stopId": "3174",
        "vehicle": {
            "id": "6990_31",
            "label": "Lintuhytti",
            "licensePlate": "SKM-931"
        }
    }
}


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, row_number
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("Exercise3").getOrCreate()

df = spark.read.csv("path/to/csv/file.csv", header=False, inferSchema=True)

df = df.withColumnRenamed("_c1", "restaurant") \
       .withColumnRenamed("_c2", "note") \
       .withColumnRenamed("_c3", "date")

df = df.withColumn("année", col("date").substr(1, 4).cast("int"))

moyenne_notes = df.groupBy("restaurant", "année") \
                  .agg(avg("note").alias("moyenne"))

window_spec = Window.partitionBy("année").orderBy(col("moyenne").desc())

ranked_restaurants = moyenne_notes.withColumn("rank", row_number().over(window_spec))

top_3_restaurants = ranked_restaurants.filter(col("rank") <= 3) \
                                       .select("année", "restaurant", "moyenne")

top_3_restaurants.orderBy("année", "moyenne", ascending=False).show()

spark.stop()
